In [45]:
#sample boilerplate code to start
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as sqlf
from functools import reduce
import marty.common.python.spark.misc.utils as misc_utils
import marty.models.lib.utils as model_utils
from marty.lib.job_manager import JobManager
import datetime
import os
import pyspark.sql.functions as sqlf
import pyspark.sql.types as sqlt
import pyspark.storagelevel as storagelevel
import yaml
import marty.common.python.io.s3 as s3_io
import marty.common.python.spark.data.stats as stat_utils
import marty.models.lib.utils as model_utils
conf = (SparkConf()
.setAppName("MAR-1057")
.set("spark.ui.showConsoleProgress", True)
.set( "spark.sql.crossJoin.enabled" , "true" )
# .set("spark.sql.parquet.fs.optimized.committer.optimization-enabled", True)
# .set("spark.driver.memory", "15g")
# .set("spark.executor.memory", "15g")
# .set("spark.storage.memoryFraction", 0)
# .setMaster('local[7]')
# .setMaster("local[7]")
# .set("spark.yarn.scheduler.heartbeat.interval-ms", 7200000)
# .set("spark.executor.heartbeatInterval", 7200000)
# .set("spark.network.timeout", 7200000)
# .set("spark.storage.memoryFraction", 0.1)
)
sc = SparkContext.getOrCreate()
print(sc)
config_path = 'marty/assignment/conf/config.yaml'
job = JobManager("MAR-1057", config_path=config_path)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/redshift/jdbc/redshift-jdbc42-1.2.37.1061.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/18 14:50:17 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


<SparkContext master=yarn appName=pyspark-shell>


In [17]:
df=job.spark.read.format("csv").option("header","true").option("sep","|").load("s3://memberacquisition-data-out-prod/assignment/preselection/slim/recent/BJs_Merkle_DP_DSProspect_INPUT_SELECTION_Campaign_20220426_23927659.txt")
#df = job.spark.csv("s3://memberacquisition-data-out-prod/assignment/preselection/slim/recent/BJs_Merkle_DP_DSProspect_INPUT_SELECTION_Campaign_20220426_23927659.txt")
df.printSchema()

root
 |-- Index_Key: string (nullable = true)
 |-- Former_Member: string (nullable = true)
 |-- Selection_Date: string (nullable = true)
 |-- Campaign_Name: string (nullable = true)
 |-- Wave: string (nullable = true)
 |-- Mail_House_Date: string (nullable = true)
 |-- Drop_Date: string (nullable = true)
 |-- in_Home_Date: string (nullable = true)
 |-- merkle_id: string (nullable = true)
 |-- cell_code: string (nullable = true)
 |-- individual_id: string (nullable = true)
 |-- household_id: string (nullable = true)
 |-- address_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- middle_initial: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- suffix: string (nullable = true)
 |-- address1: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- zip_plus4: string (nullable = true)
 |-- crrt: string (nullable = true)



In [19]:
df.select('first_name').distinct().show()

+----------+
|first_name|
+----------+
|     ADELE|
|       TIA|
|      VERA|
|    GLENNA|
|     VIJAY|
|      TONG|
|    NORVEL|
|     ELKIN|
|   BELICIA|
|  JOSELINO|
|    JAYSON|
|    CHELSI|
|    DAMIEN|
|  YULEIDYS|
|  BEATRCIE|
|     FENEL|
|    JOVANY|
|     LUNDA|
|     ANANI|
|      AHME|
+----------+
only showing top 20 rows



In [12]:
df_full = job.read("preselected_population_recent")
df_full.columns

['address_id',
 'carrier_route',
 'merkle_id',
 'zipadd',
 'household_id',
 'individual_id',
 'first_name',
 'middle_ini',
 'last_name',
 'geo_latitude',
 'geo_longitude',
 'mbr_rwd_typ',
 'mbr_typ_cd',
 'mbr_enrl_dt',
 'mbr_exp_dt',
 'mbr_rnwl_dt',
 'mbr_mfi_amt',
 'mbr_age',
 'hh_mbr_sid',
 'mbr_addr_type_flg',
 'biz_mbr',
 'biz_mbr_primary',
 'qty_current_mbr_nohh',
 'qty_past_mbr_nohh',
 'deceased_indicator',
 'prison_indicator',
 'phone_nbr',
 'mbr_stts_cd',
 'mbr_ezr_stts_ind',
 'current_mbr_address_id',
 'past_mbr_cd',
 'days_lapsed',
 'street_addr',
 'street_addr_2',
 'city',
 'state',
 'zip_frst5_cd',
 'zip_lst4_cd',
 'addr_type_flg',
 'hh_merkle_marketing_index',
 'hh_merkle_adjusted_wealth_rating',
 'hh_age_range_in_household',
 'hh_merkle_buyer_rating',
 'hh_merkle_responder_rating',
 'hh_merkle_investment_rating',
 'hh_merkle_donor_rating',
 'hh_merkle_spending_velocity',
 'hh_merkle_spending_volume',
 'hh_inferred_card_holder',
 'hh_inferred_premium_cardholder',
 'hh_merk

In [76]:
import pandas as pd
import requests

url = "https://619ca0ea68ebaa001753c9b0.mockapi.io/evaliation/dataengineer/jr/v1/users"
url = "https://619ca0ea68ebaa001753c9b0.mockapi.io/evaliation/dataengineer/jr/v1/messages"

r=requests.get(url,verify=False)
en= r.json()

/home/hadoop/.local/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '619ca0ea68ebaa001753c9b0.mockapi.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [24]:
json_rdd = sc.parallelize([r.text])

df = job.spark.read.json(json_rdd)
df.show(truncate=False)

+---------------+
|_corrupt_record|
+---------------+
|"Not found"    |
+---------------+



In [46]:

    



import datetime


    

    


    

        


In [49]:
import json
import boto3
from urllib.parse import urlparse
from pyspark.sql.types import DateType
from pyspark.sql.types import TimestampType
from pyspark.sql import Window
import pyspark.sql.functions as f
#from pyspark.sql.functions import explode_outer
#from pyspark.sql.functions import split
#from pyspark.sql.functions import col
from urllib.parse import urlparse
import pandas as pd
import numpy as np
from pyspark.sql.functions import *
import requests

config = ReadConfigFile("spark-net.yaml")
config = add_dates_to_paths(config)




Pinging the API https://619ca0ea68ebaa001753c9b0.mockapi.io/evaluation/dataengineer/jr/v1/users


/home/hadoop/.local/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '619ca0ea68ebaa001753c9b0.mockapi.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/hadoop/.local/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host '619ca0ea68ebaa001753c9b0.mockapi.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


API responded with status code 200
Pinging the API https://619ca0ea68ebaa001753c9b0.mockapi.io/evaluation/dataengineer/jr/v1/messages
API responded with status code 200
Generating JSON at /home/hadoop/usr_api.json
Generating JSON at /home/hadoop/msg_api.json


In [51]:
usr_df = job.spark.read.format('json').option("multiLine", "true").load(config["paths"]["usr_api_raw"]['path'])
msg_df = job.spark.read.format('json').option("multiLine", "true").load(config["paths"]["msg_api_raw"]['path'])

usr_df=usr_df.withColumnRenamed("id", "userId")
msg_df=msg_df.withColumnRenamed("id", "msgId")


usr_df=ConvertStringToTimeStamp(usr_df,'createdAt')
usr_df=ConvertStringToTimeStamp(usr_df,'updatedAt')
msg_df=ConvertStringToTimeStamp(msg_df,'createdAt')

#create user_sub and user_attr table
user_attr_df=usr_df.select("userId","profile.*")#.show(500,False)
user_sub_df = usr_df.select("userId",explode_outer("subscription")).select("userId","col.*")#.show(500,False)

w = Window.partitionBy('userId')
user_sub_df_slim = user_sub_df.withColumn('maxB', f.max('startDate').over(w)).where(f.col('startDate') == f.col('maxB')).drop('maxB')

usr_df = usr_df.drop("profile")
usr_df = usr_df.drop("subscription")
#adding feature column -domain
usr_df = usr_df.withColumn("domain",split(split(col("email"), "@").getItem(1),".com").getItem(0))

#temp table for qurying
msg_df.registerTempTable("msg_df")
usr_df.registerTempTable("usr_df")
user_attr_df.registerTempTable("user_attr_df")
user_sub_df.registerTempTable("user_sub_df")
user_sub_df_slim.registerTempTable("user_sub_df_slim")

#write all 4 tables to S3
write(usr_df,"user",config)
write(user_attr_df,"user_attributes",config)
write(user_sub_df,"user_subscription",config)
write(msg_df,"msg",config)
GenerateAnalyticsOutput(config)

Starting write operation for user dataset


Starting write operation for user_attributes dataset


Starting write operation for user_subscription dataset


Starting write operation for msg dataset


Starting write operation for analytics_op dataset


In [52]:
#3.1
q="SELECT count(*) as msg_cnt FROM msg_df"
job.spark.sql(q).show(500,False)
#3.2
q="""SELECT distinct(userId) as lonely_usr FROM usr_df except SELECT distinct(receiverId) FROM msg_df"""
job.spark.sql(q).show(500,False)
#3.3
q="SELECT count(*) as actv_sub_cnt from user_sub_df where status =='Active' "
job.spark.sql(q).show(500,False)
#3.4
q="""SELECT distinct(userId) as freemiuim_userId from user_sub_df_slim where status =="Active" intersect SELECT distinct(senderId) as sub_cnt from msg_df"""
job.spark.sql(q).show(500,False)

+-------+
|msg_cnt|
+-------+
|20     |
+-------+

+----------+
|lonely_usr|
+----------+
|4         |
+----------+

+------------+
|actv_sub_cnt|
+------------+
|4           |
+------------+

+----------------+
|freemiuim_userId|
+----------------+
|2               |
|5               |
|1               |
|3               |
+----------------+



In [428]:
user_sub_df_slim.withColumn("userId", lit("*** Masked ***")).show(500,False)

+--------------+------+------------------------+------------------------+------------------------+--------+
|userId        |amount|createdAt               |endDate                 |startDate               |status  |
+--------------+------+------------------------+------------------------+------------------------+--------+
|*** Masked ***|23.78 |2021-11-24T14:36:18.895Z|2022-07-13T09:14:04.001Z|2021-11-24T12:57:48.724Z|Active  |
|*** Masked ***|89.71 |2021-11-24T14:40:24.257Z|2022-11-23T12:41:28.319Z|2021-11-24T11:22:33.265Z|Active  |
|*** Masked ***|43.18 |2021-11-24T16:58:46.581Z|2022-09-15T06:05:59.630Z|2021-11-24T05:12:49.301Z|Active  |
|*** Masked ***|3.51  |2021-11-24T02:07:08.482Z|2021-12-10T20:22:36.132Z|2021-11-24T12:47:33.246Z|Inactive|
|*** Masked ***|88.60 |2021-11-23T18:57:20.540Z|2022-03-03T09:47:26.916Z|2021-11-24T18:04:41.908Z|Active  |
+--------------+------+------------------------+------------------------+------------------------+--------+



b'ProUkE0wwl8q2JR7J2DDO6NMkwWnGcufWaB5sVFjL40='


InvalidToken: 

In [120]:
def generate_key():
    """
    Generates a key and save it into a file
    """
    key = Fernet.generate_key()
    with open("secret.key", "wb") as key_file:
        key_file.write(key)
generate_key()

def load_key():
    """
    Load the previously generated key
    """
    return open("secret.key", "rb").read()

def ConvertStringToTimeStamp(spark_df,ts_col):
    spark_df = spark_df.withColumn("temp_ts_col",spark_df[ts_col].cast(TimestampType()))
    spark_df = spark_df.drop(ts_col)
    spark_df=spark_df.withColumnRenamed("temp_ts_col", ts_col)
    return spark_df

def encrypt_message(message):
    """
    Encrypts a message
    """
    #key = load_key()
    #print(key)
    import cryptography

    key = b'SgC_PuQOhFINn8XkKhnWMOKtWTSl8RnUXchTbeCz1XS='

    encoded_message = message.encode()
    f = Fernet(key)
    encrypted_message = f.encrypt(encoded_message)

    print(encrypted_message)
    return encrypted_message
    
def decrypt_message(encrypted_message):
    """
    Decrypts an encrypted message
    """
    #key = load_key()
    key = b'SgC_PuQOhFINn8XkKhnWMOKtWTSl8RnUXchTbeCz1XS='

    f = Fernet(key)
    decrypted_message = f.decrypt(encrypted_message)

    print(decrypted_message.decode())

aa= encrypt_message("shubham is my name")
decrypt_message(aa)

b'gAAAAABihRuceiqrf7KGyGyzlHBrYLfY3yly6aWZ6wGNis9aogjrbGqeONZyQxSjhcjyAELaCbsvz5bbXKeU4ODeCkClItJl2MC3ZswtjM5b1KnFLfowG5k='
shubham is my name


In [82]:
!pip3 install cryptography
!pip3 install paramiko

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://nexus.bjs.com/repository/pip-registry/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://nexus.bjs.com/repository/pip-registry/simple


In [117]:
!python -m pip install cryptography

/usr/bin/python: No module named pip


In [121]:
import cryptography
from cryptography.fernet import Fernet

from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType

encrypt_message_fn = udf(lambda x:encrypt_message(x),StringType() )
r= usr_df.withColumn("fname_en",encrypt_message_fn(col("firstname")))
import cryptography
r.show()

22/05/18 16:15:28 WARN TaskSetManager: Lost task 0.0 in stage 86.0 (TID 4206) (ip-10-13-53-162.bjw2k.asg executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/mnt/yarn/usercache/hadoop/appcache/application_1652487863319_0004/container_1652487863319_0004_01_000002/pyspark.zip/pyspark/worker.py", line 588, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/mnt/yarn/usercache/hadoop/appcache/application_1652487863319_0004/container_1652487863319_0004_01_000002/pyspark.zip/pyspark/worker.py", line 447, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/mnt/yarn/usercache/hadoop/appcache/application_1652487863319_0004/container_1652487863319_0004_01_000002/pyspark.zip/pyspark/worker.py", line 249, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/mnt/yarn/usercache/hadoop/appcache/application_165248786

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/mnt/yarn/usercache/hadoop/appcache/application_1652487863319_0004/container_1652487863319_0004_01_000002/pyspark.zip/pyspark/worker.py", line 588, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/mnt/yarn/usercache/hadoop/appcache/application_1652487863319_0004/container_1652487863319_0004_01_000002/pyspark.zip/pyspark/worker.py", line 447, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/mnt/yarn/usercache/hadoop/appcache/application_1652487863319_0004/container_1652487863319_0004_01_000002/pyspark.zip/pyspark/worker.py", line 249, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/mnt/yarn/usercache/hadoop/appcache/application_1652487863319_0004/container_1652487863319_0004_01_000002/pyspark.zip/

In [96]:
encrypt_message_fn(col("firstname")).show()

TypeError: 'Column' object is not callable